In [11]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [ ]:
# typing 모듈의 Dict : 딕셔너리 자료형의 타입 힌트를 제공 (예: Dict[str, int])
from typing import Dict

# InMemoryChatMessageHistory : 대화 내용을 메모리에 저장하는 클래스
# (대화를 파일이나 DB에 저장하지 않고 임시로 메모리에 저장할 때 사용)
from langchain_core.chat_history import InMemoryChatMessageHistory

# RunnableWithMessageHistory : 모델 실행 시 이전 대화 기록을 함께 전달하도록 도와주는 클래스
# (예: 사용자의 이전 질문과 답변을 기억한 상태로 대화를 이어가기 가능)
from langchain_core.runnables.history import RunnableWithMessageHistory

# ChatPromptTemplate : 대화형 프롬프트를 만들 때 사용하는 템플릿 클래스
# (AI가 이해할 수 있는 대화 형식의 입력 구조를 만드는 역할)
# MessagesPlaceholder : 프롬프트 안에 “대화 기록 자리”를 미리 만들어두는 기능
# (이전 대화 내용을 넣을 위치를 표시)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# StrOutputParser : 모델이 생성한 출력 결과를 문자열 형태로 단순하게 반환하는 파서
# (AI의 응답을 그대로 텍스트 형태로 받아볼 때 사용)
from langchain_core.output_parsers import StrOutputParser


In [ ]:
# ChatPromptTemplate.from_messages() : 여러 메시지를 조합해서 하나의 대화 프롬프트를 만드는 메서드
prompt = ChatPromptTemplate.from_messages([
    # system 역할 : AI의 전체적인 성격이나 말투를 설정
    ("system", "너는 AI 도우미야, 간략하게 그냥 읍답하도록 해"),

    # MessagesPlaceholder : 이전 대화 기록(history)을 저장하고 불러오는 자리
    # (사용자와 AI가 주고받은 이전 대화를 여기에 자동으로 채워넣음)
    MessagesPlaceholder(variable_name="history"),

    # user 역할 : 사용자의 실제 질문이 들어오는 부분
    ("user", "{question}")
])

# 프롬프트 전체를 확인 (AI가 어떤 구조로 입력을 받을지 보여줌)
prompt


ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [15]:
chain = prompt | model | StrOutputParser()

In [ ]:
# 2. 대화 내용 저장소 만들기

# 여러 사용자의 대화 내용을 저장하기 위한 딕셔너리 (session_id별로 구분)
# 예: {"user1": history1, "user2": history2}
stores: Dict[str, InMemoryChatMessageHistory] = {}

# 특정 사용자의 session_id를 받아서, 그 사람의 대화 기록을 가져오는 함수
def get_store(session_id: str):
    # 만약 해당 session_id가 아직 저장소에 없다면 (즉, 처음 대화하는 사용자라면)
    if session_id not in stores:
        # 새로운 대화 기록 객체를 만들어 저장
        stores[session_id] = InMemoryChatMessageHistory()
    # 해당 사용자의 대화 기록(메시지 히스토리)을 반환
    return stores[session_id]

In [ ]:
# RunnableWithMessageHistory : LangChain에서 "대화 히스토리(이전 대화 내용)"를 자동으로 관리해주는 도우미 클래스
with_history = RunnableWithMessageHistory(
    chain,                             # 실제 실행할 체인 (Prompt + Model + Parser 등으로 구성된 흐름)
    lambda session_id: get_store(session_id),  # 세션 ID로 각 사용자의 대화 기록을 가져오는 함수
    input_messages_key="question",     # 사용자 입력(질문)이 들어가는 변수 이름
    history_messages_key="history"     # 이전 대화 기록을 불러올 변수 이름
)

In [ ]:
# cfg : 대화할 사용자를 지정 (여기서는 "user-123" 세션)
cfg = {"configurable": {"session_id": "user-123"}}

# with_history.invoke() : 질문을 AI에 보내고, 이전 대화(history)와 함께 처리
# "question" : 사용자가 AI에게 묻는 내용
# config=cfg : 어느 사용자의 대화 기록을 사용할지 지정
result = with_history.invoke({"question": "내가 좋아 하는 폰은 갤럭시야"}, config=cfg)

# AI가 대답한 결과를 출력
print(result)


갤럭시 좋은 선택이야! 어떤 모델 좋아해?


In [19]:
result = with_history.invoke({"question" : "그 장점만 요약해봐"}, config= cfg)
print(result)

갤럭시 장점 요약:
- 뛰어난 디스플레이 품질 (AMOLED)
- 강력한 성능과 빠른 처리 속도
- 다양한 카메라 기능과 우수한 사진 품질
- 방수·방진 지원
- 확장 가능한 저장 공간
- 삼성 생태계와의 높은 호환성


In [20]:
result = with_history.invoke({"question" : "아이폰보다 좋아?"}, config= cfg)
print(result)

용도에 따라 다르지만, 갤럭시는 커스터마이징과 확장성, 디스플레이에서 강점이 있어. 아이폰은 보안과 최적화, 앱 생태계가 뛰어나.


In [21]:
result = with_history.invoke({"question" : "너의 개인적인 견해를 말해"}, config= cfg)
print(result)

나는 개인적인 견해가 없지만, 사용자 취향과 필요에 맞게 선택하는 게 가장 좋아.


In [ ]:
# stores["user-123"].messages 
# "user-123" 세션에 저장된 대화 기록 전체를 확인
# 각 메시지(질문/답변)가 순서대로 리스트 형태로 저장되어 있음
stores["user-123"].messages

[HumanMessage(content='내가 좋아 하는 폰은 갤럭시야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 좋은 선택이야! 어떤 모델 좋아해?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='그 장점만 요약해봐', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 장점 요약:\n- 뛰어난 디스플레이 품질 (AMOLED)\n- 강력한 성능과 빠른 처리 속도\n- 다양한 카메라 기능과 우수한 사진 품질\n- 방수·방진 지원\n- 확장 가능한 저장 공간\n- 삼성 생태계와의 높은 호환성', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='아이폰보다 좋아?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='용도에 따라 다르지만, 갤럭시는 커스터마이징과 확장성, 디스플레이에서 강점이 있어. 아이폰은 보안과 최적화, 앱 생태계가 뛰어나.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='너의 개인적인 견해를 말해', additional_kwargs={}, response_metadata={}),
 AIMessage(content='나는 개인적인 견해가 없지만, 사용자 취향과 필요에 맞게 선택하는 게 가장 좋아.', additional_kwargs={}, response_metadata={})]